In [1]:
import yaml 

import os 
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
import yaml
import os 
import torch
from  torch.utils.data import Dataset, DataLoader 
import sys
import torch
import torch.nn as nn
from torch.nn import Parameter
import math
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
from tqdm.notebook import tqdm
import torch.nn as nn
import time

# module_path = "C:\\Users\\Dung\\3D Objects\\Air-Quality-Prediction-Benchmark"
# if module_path not in sys.path:
#     sys.path.append(module_path)

import os, psutil
from util import model as model_utils
from util.single_loader import get_data_array, get_dataloader
from util.early_stop import EarlyStopping, EarlyStoppingReachAccuracy
from util.torch_util import weight_init
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from tqdm import tqdm
import wandb
import argparse
import time
from util.loss import r2_loss


ModuleNotFoundError: No module named 'util'

In [2]:
def get_data_array(config, target_station):
    file_path = config['data_dir']

    file_gauge = file_path + 'gauges/'

    list_input_ft = config['input_features']
    scaler = MinMaxScaler()
    df = pd.read_csv(file_gauge  + f"{target_station}.csv")[list_input_ft]
    arr = df.iloc[:,:].astype(np.float32).values

    scaler.fit(arr)
    transformed_data = scaler.transform(arr)
    return transformed_data,  scaler

def get_dataloader(data,  config, train_ratio = 0.3):
    len_dataset = len(data)  
    train_pct = config['train_size']
    valid_pct = config['valid_size']
    test_pct = config['test_size']

    train_data = data[:int(len_dataset * train_pct * train_ratio)]
    valid_data = data[int(len_dataset * (1 - test_pct - valid_pct)): int(len_dataset * (1 - test_pct))]
    test_data = data[int(len_dataset * (1 - test_pct)):]
    train_dataset = AQDataset(
        data= train_data, 
        config=config
    )
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'],
        shuffle=False, 
        drop_last=True
    )
    validation_dataset = AQDataset(
        data=valid_data, 
        config=config
    )
    valid_dataloader = DataLoader(
        validation_dataset,
        batch_size=config['batch_size'],
        shuffle=False, 
        drop_last=True
    )

    test_dataset = AQDataset(
        data=test_data,
        config=config
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=config['batch_size'],
        shuffle=False, 
        drop_last=True
    )

#     if args.data_splitting == 'hold-out':
#         train_dataset = AQDataset(
#             data= train_data, 
#             config=config
#         )
#         train_dataloader = DataLoader(
#             train_dataset, 
#             batch_size=config['batch_size'],
#             shuffle=True, 
#             drop_last=True
#         )
#         validation_dataset = AQDataset(
#             data=valid_data, 
#             config=config
#         )
#         valid_dataloader = DataLoader(
#             validation_dataset,
#             batch_size=config['batch_size'],
#             shuffle=False, 
#             drop_last=True
#         )

#         test_dataset = AQDataset(
#             data=test_data,
#             config=config
#         )
#         test_dataloader = DataLoader(
#             test_dataset,
#             batch_size=config['batch_size'],
#             shuffle=False, 
#             drop_last=True
#         )
    
#     elif args.data_splitting == 'time-series-cross-validation':
#         pass
#     elif args.data_splitting == 'blocking-cross-validation':
#         pass
    return train_dataloader, valid_dataloader, test_dataloader

class AQDataset(Dataset):
    def __init__(self, data, config):
        super().__init__()
        self.data = data 
        self.config = config 
        self.input_len =  self.config['window_size']
        self.output_len = self.config['output_size']

        self.input_feats =  self.config["input_features"]
        self.target_feat = self.config['target_features']
        self.target_ft_idx = self.config['input_features'].index(self.config['target_features'][0])
    
    def __getitem__(self, index: int):
        x = self.data[
            index: index + self.input_len, 
            :
        ]
        y = self.data[
            index + self.input_len: index + self.input_len + self.output_len, 
            self.target_ft_idx
        ]
        return {
            'x': torch.from_numpy(x),
            'y': torch.from_numpy(y)
        }

    def __len__(self) -> int:
        return self.data.shape[0]  - self.input_len - self.output_len +1


In [3]:
config_path = '../../config/magan.yml'
with open(config_path, 'r', encoding = "utf-8") as f:
    config= yaml.safe_load(f)
data, scaler = get_data_array(config,"房山") 
train_dataloader, valid_dataloader, test_dataloader = get_dataloader(data=data, config=config)

for data in train_dataloader:
    input = data['x']
    print(input.shape)
    y_fake = input[:,:,0]
    print(y_fake)
    break


torch.Size([64, 24, 5])
tensor([[0.2500, 0.2683, 0.2988,  ..., 0.3720, 0.3476, 0.3476],
        [0.2683, 0.2988, 0.2378,  ..., 0.3476, 0.3476, 0.3323],
        [0.2988, 0.2378, 0.2317,  ..., 0.3476, 0.3323, 0.3171],
        ...,
        [0.0366, 0.0488, 0.0305,  ..., 0.0793, 0.0976, 0.0762],
        [0.0488, 0.0305, 0.0122,  ..., 0.0976, 0.0762, 0.0549],
        [0.0305, 0.0122, 0.0610,  ..., 0.0762, 0.0549, 0.0793]])


In [4]:
class Encoder(nn.Module):
  def __init__(self,config,device): #,m,T,n
    super(Encoder,self).__init__()
    self.hidden_size = config['hidden_size']
    self.batch_size = config['batch_size']
    self.window_size = config['window_size']
    self.driving_series = config['driving_series']
    self.device = device
    
    #input attention
    self.Ve = Parameter(torch.Tensor(self.window_size)).to(self.device)
    self.We = Parameter(torch.Tensor(self.window_size,2*self.hidden_size)).to(self.device)
    self.Ue = Parameter(torch.Tensor(self.window_size,self.window_size)).to(self.device)
    self.LSTM1 = nn.LSTM(self.driving_series,self.hidden_size,1,batch_first = True).to(self.device)

    #self attention
    self.Wg = Parameter(torch.Tensor(self.hidden_size,self.batch_size)).to(self.device)
    self.Wa = Parameter(torch.Tensor(self.batch_size,self.hidden_size)).to(self.device)
    self.bg = Parameter(torch.Tensor(self.hidden_size,self.driving_series)).to(self.device)
    self.ba = Parameter(torch.Tensor(self.batch_size,self.driving_series)).to(self.device)
    self.LSTM2 = nn.LSTM(self.driving_series,self.hidden_size,1,batch_first = True).to(self.device)

    #function
    self.tanh = nn.Tanh()
    self.sigmoid = nn.Sigmoid()
    self.softmax = nn.Softmax(dim=1)

    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)
  
  def forward(self,x,h,s):
      batch, T,n = x.size()
      h1 = (h,s)
      h2 = (h,s)
      input_attention = torch.zeros(batch,n,T).to(self.device)
      self_attention = torch.zeros(batch,T,n).to(self.device)
    # for i in range(batch):
      for j in range(n):
# #         print("H",h.shape)
# #         print('s', s.shape)
#         print("Concat", torch.cat((h,s),2).shape)
#         print(self.We.shape)

# #         print(self.We @ (torch.cat((h,s),2))
        print(self.We @ torch.cat((h,s),2).permute((1,2,0)))
#         print(self.We@ (torch.cat((h,s),2).transpose(2,1)))
        e_k = self.Ve.T @ self.tanh(self.We@ (torch.cat((h,s),2).transpose(2,1) + self.Ue@x[:,:,j].T))
#         print(e_k.shape)
        a_k = self.softmax(e_k)
        # print(a_k.shape)
        input_attention[:,j,:] = a_k @ x[:,:,j]
      for d in range(T):
        g_t = self.tanh(self.Wg @ x[:,d,:] + self.bg)
        a_t = self.sigmoid(self.Wa @ g_t + self.ba)
        # print(a_t.shape)
        # print(self_attention[:,d,:].shape)
        self_attention[:,d,:] = a_t * x[:,d,:]
      print(self_attention.sha, input_attention)
      _,h1 = self.LSTM1(input_attention, h1)
      _,h2 = self.LSTM2(self_attention.transpose(2,1),h2)
      return torch.cat((h1[0],h2[0]),0), h1
    
  def init_hidden(self):
    h = torch.zeros(1,self.batch_size,self.hidden_size).to(self.device)
    c = torch.zeros(1,self.batch_size,self.hidden_size).to(self.device)
    return h,c

#generator
class Generator(nn.Module):
  def __init__(self,config,device): #p,T,n,m,k,w
    super(Generator,self).__init__()
    self.hidden_size = config['hidden_size']
    self.num_filter = config['num_filter']
    self.batch_size = config['batch_size']
    self.window_size = config['window_size']
    self.output_size = 1
    self.hidden_size = config['hidden_size']
    self.kernel_size = config['kernel_size']
    self.device = device
    #Parameter define
    self.vd = Parameter(torch.Tensor(self.hidden_size)).to(self.device)
    self.Wd = Parameter(torch.Tensor(self.hidden_size,2*self.hidden_size)).to(self.device)
    self.Ud = Parameter(torch.Tensor(self.hidden_size,self.hidden_size)).to(self.device)
#     self.w = Parameter(torch.Tensor(1,self.hidden_size + 1)).to(self.device)
#     self.b = Parameter(torch.Tensor(self.output_size,1)).to(self.device)
    self.linear = nn.Linear(self.hidden_size + 1,1).to(self.device)
    #function
    self.tanh = nn.Tanh()
    self.softmax = nn.Softmax(dim=1)
    self.relu = nn.LeakyReLU(0.2, inplace = True)

    self.conv = nn.Conv1d(2,self.num_filter,self.kernel_size).to(self.device)
    self.LSTM = nn.LSTM(self.output_size,self.hidden_size,1,batch_first = True).to(self.device)

    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)

  def forward(self,Z,d,s,y_real):
    # print(Z.shape)
    H = self.relu(self.conv(Z.transpose(1,0)))
#     print("H", H)
    ct = torch.zeros(self.batch_size,self.hidden_size).to(self.device)
    for i in range(H.shape[1]):
      l = self.vd.T @ self.tanh(self.Wd @ (torch.cat((d,s),2).transpose(2,1)) + self.Ud @ (H[:,i,:].T))
#       print("l",l)

      B = self.softmax(l)
      ct += B@H[:,i,:]
      y_real = y_real
#       print("Ct", ct.shape)
#       print("Y real",y_real.shape)
# #       y_fake = self.w @ torch.cat((y_real,ct),1) + self.b
#       concat = torch.cat((y_real.unsqueeze(-1),ct),-1)
      concat = torch.cat((y_real,ct),-1)
      y_fake = self.linear(concat)
#       print(y_fake)
      _,(d,s) =  self.LSTM(y_fake.view(self.batch_size,1,self.output_size),(d,s))
    return y_fake.view(self.batch_size,self.output_size),(d,s)

  def init_hidden(self):
    h = torch.zeros(1,self.batch_size,self.hidden_size).to(self.device)
    c = torch.zeros(1,self.batch_size,self.hidden_size).to(self.device)
    return h,c

#discriminator
class Discriminator(nn.Module):
  def __init__(self,config, device):
    super(Discriminator,self).__init__()
    self.output_size = 1
    self.batch_size = config['batch_size']
    self.device = device
    self.conv1 = nn.Conv1d(self.output_size,128,1).to(self.device)
    self.conv2 = nn.Conv1d(128,256,1).to(self.device)
    self.conv3 = nn.Conv1d(256,512,1).to(self.device)
    self.linear = nn.Linear(512,self.output_size).to(self.device)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x):
    x = x.view(1,self.output_size,self.batch_size)
    x = self.conv3(self.conv2(self.conv1(x)))
    print(x)
    x = self.linear(x.view(1,self.batch_size,512))
    return self.sigmoid(x)

class MARNN(nn.Module):
  def __init__(self,config,device):
    super(MARNN,self).__init__()
    self.encode = Encoder(config,device)
    self.decode = Generator(config,device)

  def forward(self,x,y_real):
    he,se = self.encode.init_hidden()
    hd,sd = self.decode.init_hidden()
    Z,_ = self.encode(x,he,se)
    y_fake,h = self.decode(Z,hd,sd,y_real)
    return y_fake

class MAGAN(nn.Module):
  def __init__(self,config,device):
    super(MAGAN,self).__init__()
    self.marnn = MARNN(config,device)
    self.dis = Discriminator(config, device)

  def forward(self,x,y_real):
    out_f = self.marnn(x,y_real)
    out = self.dis(out_f)
    return out

In [5]:
class TimeDistributedCNN(nn.Module):

    def __init__(self, module, batch_first=False):
        super(TimeDistributedCNN, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        ''' x size: (batch_size, time_steps, num_channels, input_size) '''
        batch_size, time_steps, num_channels, input_size = x.size()
        c_in = x.view(batch_size * time_steps, num_channels, input_size)
        c_out = self.module(c_in)
#         print(c_out.shape)
        r_in = c_out.view(batch_size, time_steps, c_out.shape[1], c_out.shape[2])
        if self.batch_first is False:
            r_in = r_in.permute(1, 0, 2)

        return r_in


In [6]:
class AdditiveAttention(nn.Module):
  def __init__(self, key_size, query_size, num_hiddens, dropout):
    super(AdditiveAttention, self).__init__(**kwargs)
    self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
    self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
    self.w_v = nn.Linear(num_hiddens, 1, bias=False)
    self.dropout = nn.Dropout(dropout)

  def forward(self, queries, keys, values):
    queries, keys = self.W_q(queries), self.W_k(keys)
    # After dimension expansion, shape of `queries`: (`batch_size`, no. of
    # queries, 1, `num_hiddens`) and shape of `keys`: (`batch_size`, 1,
    # no. of key-value pairs, `num_hiddens`). Sum them up with
    # broadcasting
    features = queries.unsqueeze(2) + keys.unsqueeze(1)
    features = torch.tanh(features)
    # There is only one output of `self.w_v`, so we remove the last
    # one-dimensional entry from the shape. Shape of `scores`:
    # (`batch_size`, no. of queries, no. of key-value pairs)
    scores = self.w_v(features).squeeze(-1)
    self.attention_weights = nn.functional.softmax(scores, dim=-1)
    # Shape of `values`: (`batch_size`, no. of key-value pairs, value
    # dimension)
    return torch.bmm(self.dropout(self.attention_weights), values)

In [134]:
class Encoder(nn.Module):
  def __init__(self,config,device): #,m,T,n
    super(Encoder,self).__init__()
    self.hidden_size = config['hidden_size']
    self.window_size = config['window_size']
    self.driving_series = config['driving_series']
    self.device = device
    
    self.input_lstm = nn.LSTM(self.driving_series,self.hidden_size,1,batch_first = True).to(self.device)

#     self.We = nn.Linear(self.hidden_size * 2, self.window_size, bias = False).to(self.device)
#     self.Ue = nn.Linear(self.window_size, self.window_size, bias = False).to(self.device)
#     self.Ve = nn.Linear(self.window_size, 1, bias = False).to(self.device)
    self.We = nn.Linear(self.hidden_size * 2, self.hidden_size, bias = False).to(self.device)
    self.Ue = nn.Linear(self.driving_series, self.hidden_size, bias = False).to(self.device)
    self.Ve = nn.Linear(self.hidden_size, 1, bias = False).to(self.device)

    self.LSTM1 = nn.LSTM(self.driving_series,self.hidden_size,1,batch_first = True).to(self.device)

    #self attention
#     self.Wg = nn.Linear(in_features = self.driving_series, out_features  = self.hidden_size).to(self.device)
#     self.Wa = nn.Linear(in_features = self.hidden_size, out_features  = self.driving_series).to(self.device)
    self.Wg = nn.Linear(in_features = self.driving_series, out_features  = self.hidden_size).to(self.device)
    self.Wa = nn.Linear(in_features = self.hidden_size, out_features  = self.driving_series).to(self.device)

    self.LSTM2 = nn.LSTM(self.driving_series,self.hidden_size,1,batch_first = True).to(self.device)

    #function
    self.tanh = nn.Tanh()
    self.sigmoid = nn.Sigmoid()
    self.softmax = nn.Softmax(dim=1)

    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)
  
  def forward(self,x):
      batch_size, T,n = x.size()
      h, s = self.init_hidden(batch_size)
      h1 = (h,s)
      h2 = (h,s)
      input_attention = torch.zeros(batch_size,T,n).to(self.device)
      
      # Calculate input attention
#       output, (_,s) = self.input_lstm(x)
      # Initialize the hidden state and cell state (1, batch_size, hidden_size)
      (h,s) = (h,s)
      for t in range(T):
        #x_t: input at time step t (batch_size, input_features)
        x_t = x[:,t,:].unsqueeze(1)
        # Concatenate the hidden state and cell state [h,s] (batch_size, hidden_size * 2)
        h_s = torch.cat((h.squeeze(0),s.squeeze(0)),-1)
        h_s = h_s.unsqueeze(1).repeat(1, self.window_size, 1)
        # Calculate the attention weights
        e_t = self.Ve(self.tanh(self.We(h_s) + self.Ue(x)))
        a_t = self.softmax(e_t.squeeze(-1)).unsqueeze(1)
        # Calculate input attention
        input_attention[:,t,:] = (a_t @ x).squeeze(1)
        # Pass through an lstm layer to get the current hidden state and cell state
        _,(h,s) = self.input_lstm(x_t, (h,s))
        
      # Calculate self attention
      a = []
      for d in range(T):
        # input at time step t (batch_size, input_features)
        xt = x[:,d,:]
        g_t = self.tanh(self.Wg(xt))
        a_t = self.sigmoid(self.Wa(g_t))
        a.append(a_t)
      a = torch.stack(a, dim = 2)
      self_attention = a * x.transpose(2,1)
        
      # Pass input attention and self attention through 2 lstms
      output_1,h1 = self.LSTM1(input_attention, h1)
      output_2,h2 = self.LSTM2(self_attention.transpose(2,1),h2)
      
      # Concatenate 2 lstm outputs as latent space Z (batch_size, time_steps,hidden_size * 2)
      z = torch.cat((output_1, output_2), dim = -1)
      return z
    
  def init_hidden(self, batch_size):
    h = torch.zeros(1,batch_size,self.hidden_size).to(self.device)
    c = torch.zeros(1,batch_size,self.hidden_size).to(self.device)
    return h,c

#generator
class Generator(nn.Module):
  def __init__(self,config,device): #p,T,n,m,k,w
    super(Generator,self).__init__()
    self.hidden_size = config['hidden_size']
    self.num_filter = config['num_filter']
#     self.batch_size = config['batch_size']
    self.window_size = config['window_size']
    self.output_size = config['output_size']
    self.hidden_size = config['hidden_size']
    self.kernel_size = config['kernel_size']
    self.device = device
    #Parameter define
#     self.Wd = nn.Linear(self.hidden_size * 2, self.hidden_size, bias = False).to(self.device)
#     self.Ud = nn.Linear(self.hidden_size, self.hidden_size, bias = False).to(self.device)
#     self.vd = nn.Linear(self.hidden_size, 1, bias = False).to(self.device)
#     self.output_linear = nn.Linear((self.hidden_size) + 1, 1).to(self.device)
    self.Wd = nn.Linear(self.hidden_size * 2 , self.hidden_size, bias = False).to(self.device)
    self.Ud = nn.Linear(self.hidden_size, self.hidden_size, bias = False).to(self.device)
    self.vd = nn.Linear(self.hidden_size, 1, bias = False).to(self.device)
    self.output_linear = nn.Linear((self.hidden_size ) + 1, 1).to(self.device)

    #function
    self.tanh = nn.Tanh()
    self.softmax = nn.Softmax(dim=-1)
    self.relu = nn.ReLU()

    self.conv = nn.Conv1d(1,self.num_filter,self.kernel_size, padding = 'same').to(self.device)
    self.distributed_conv = TimeDistributedCNN(self.conv, batch_first = True)
    self.LSTM = nn.LSTM(1,self.hidden_size,1,batch_first = True).to(self.device)
    self.conv_linear = nn.Linear(self.num_filter * self.hidden_size * 2, self.hidden_size).to(self.device)

    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1.0 / math.sqrt(self.hidden_size)
    for weight in self.parameters():
      weight.data.uniform_(-stdv, stdv)

  def forward(self,Z,y_real):
    batch_size = Z.shape[0]
    # Pass the latent space through time distributed conv
    Z = Z.unsqueeze(2)
    H = self.relu(self.distributed_conv(Z))
    # H shape (batch_size,time_steps, num_filters, hidden_size * 2)
    # Reshape H to (batch_size, time_steps, hidden_size)
    
    H = H.view(H.shape[0], H.shape[1], H.shape[2] * H.shape[3])
    H = self.relu(self.conv_linear(H))
    
    # Attention and predict y
    d, s = self.init_hidden(batch_size)
    previous_y = y_real.unsqueeze(-1)
    predictions = []
    for t in range(self.output_size):
        # Concat previous hidden state and cell state
        d_s = torch.cat((d,s),2).squeeze(0)
        d_s = d_s.unsqueeze(1).repeat(1, self.window_size, 1)
        #Calculate attention
        l_t = self.vd(self.tanh(self.Wd(d_s) + self.Ud(H)))
        b_t = self.softmax(l_t.squeeze(-1))
        b_t = b_t.unsqueeze(1)
        # Calculate context vector
        c_t = (b_t @ H).squeeze(1)
        # Calculate prediction at time step t
        y_t = self.output_linear(torch.cat((c_t, previous_y), dim = 1))
        predictions.append(y_t)
        y_t = y_t.unsqueeze(-1)
        # Pass through an LSTM layer to get current hidden state, cell state
        _,(d,s) = self.LSTM(y_t, (d,s))
    predictions = torch.cat(predictions, dim = 1)
    return predictions

  def init_hidden(self, batch_size):
    h = torch.zeros(1,batch_size,self.hidden_size).to(self.device)
    c = torch.zeros(1,batch_size,self.hidden_size).to(self.device)
    return h,c

#discriminator
class Discriminator(nn.Module):
  def __init__(self,config, device):
    super(Discriminator,self).__init__()
    self.output_size = config['output_size']
#     self.batch_size = config['batch_size']
    self.device = device
    self.kernel_size = config['kernel_size']
    self.conv1 = nn.Sequential(nn.Conv1d(1,32,self.kernel_size, padding = 'same').to(self.device),nn.LeakyReLU())
    self.conv2 = nn.Sequential(nn.Conv1d(32,64,self.kernel_size, padding = 'same').to(self.device),nn.LeakyReLU())
    self.conv3 = nn.Sequential(nn.Conv1d(64,128,self.kernel_size, padding = 'same').to(self.device),nn.LeakyReLU())
    self.linear = nn.Linear(128 * self.output_size,1).to(self.device)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x):
    batch_size = x.shape[0]
    x = x.unsqueeze(1)
    x = self.conv3(self.conv2(self.conv1(x)))
    x = self.linear(x.view(batch_size,128 * self.output_size))
    return self.sigmoid(x)

class MARNN(nn.Module):
  def __init__(self,config,device):
    super(MARNN,self).__init__()
    self.encode = Encoder(config,device)
    self.decode = Generator(config,device)

  def forward(self,x,y_real):
#     he,se = self.encode.init_hidden()
#     hd,sd = self.decode.init_hidden()
    Z = self.encode(x)
    y_fake = self.decode(Z,y_real)
    return y_fake

class MAGAN(nn.Module):
  def __init__(self,config,device):
    super(MAGAN,self).__init__()
    self.marnn = MARNN(config,device)
    self.encoder = self.marnn.encode
    self.generator = self.marnn.decode
    self.dis = Discriminator(config, device)

  def forward(self,x,y_real):
    out_f = self.marnn(x,y_real)
    out = self.dis(out_f)
    return out

In [135]:
config_path = '../../config/magan_test.yml'
with open(config_path, 'r', encoding = "utf-8") as f:
    config= yaml.safe_load(f)
data, scaler = get_data_array(config,"房山") 
train_dataloader, valid_dataloader, test_dataloader = get_dataloader(data=data, config=config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for data in train_dataloader:
    input = data['x'].to(device)
    target = data['y'].to(device)
    previous_y = input[:,-1,0].to(device)
    
    model = MAGAN(config, device).to(device)
    predictions = model.marnn(input, previous_y)
    print(predictions.shape)
    d_out_fake = model.dis(predictions)
    d_out_real = model.dis(target)
    print(d_out_fake)
    print(d_out_real)

    break



torch.Size([128, 1])
tensor([[0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
        [0.5039],
       

In [136]:
1536 / 24

64.0

In [137]:
import torch.nn.init as init
def weight_init(m):
    '''
    Usage:
        model = Model()
        model.apply(weight_init)
    '''
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.LSTMCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRU):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRUCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)

In [170]:
class MAGANSupervisor():
    def __init__(self, config, target_station, device):
        self._epochs = config["epochs"]
        self._lr = config["lr"]
        self.window_size = config["window_size"]

        self._optimizer = config['optimizer']
        self.optimizer = config['optimizer']

        self.input_size = config['window_size']
        self.input_dim = config['driving_series']

        self._hidden_size = config['hidden_size']
        self._output_size = config['output_size']

        self._batch_size = config['batch_size']

        # Data
        self.target_station = target_station
        transformed_data, scaler = get_data_array(config, target_station)
        self._scaler = scaler 
        self.true_labels = torch.ones(self._batch_size,1).to(device)
        self.fake_labels = torch.zeros(self._batch_size,1).to(device)
        self.device = device
        self._base_dir = os.path.join(config['base_dir'])
        self._weights_path = os.path.join(self._base_dir, "best.pth")

        train_dataloader, valid_dataloader, test_dataloader = get_dataloader(data=transformed_data, config=config)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.test_dataloader = test_dataloader
        first_it = next(iter(train_dataloader))   
        X, y = first_it['x'], first_it['y']

        self.config = config

        # Model
        self._model = MAGAN(config, device).to(self.device)
        self._model.apply(weight_init)

        
        self._es = EarlyStopping(
            patience=self.config['patience'],
            verbose=True,
            delta=0.0,
            path=self._weights_path 

        )

        self._es_until_reach_accuracy = EarlyStoppingReachAccuracy(
            patience=self.config['patience'],
            verbose=True,
            delta=0.0,
            path=self._weights_path   

        )
        self.train_ratio = 0.3
        self.stop_until_reach_accuracy = False
#         if args.experimental_mode == 'stop_until_reach_accuracy':
#             self.stop_until_reach_accuracy = True

    def train(self):
        torch.autograd.set_detect_anomaly(True)
        val_losses = []
        clip = 5
        model = self._model
        criterion = torch.nn.MSELoss()
        dis_criterion = torch.nn.BCELoss()
        es = EarlyStopping(
            patience=100,
            verbose=True,
            delta=0.0,
            path=self._weights_path    
        )
        total_train_time = 0.0
        if self.optimizer == 'adam':
            optimizer_generator = torch.optim.Adam(model.generator.parameters(), lr=self._lr)
            optimizer_dis = torch.optim.Adam(model.dis.parameters(), lr=self._lr)
            optimizer_marnn_t = torch.optim.Adam(model.marnn.parameters(), lr=self._lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_marnn_t, mode='min',
            factor=0.1, patience=3, threshold=0.0001, threshold_mode='abs')

        # Train the model
        for epoch in range(self._epochs):
            if not es.early_stop:
                epoch_trainD_loss = 0 
                epoch_trainG_loss = 0
                epoch_train_marnn_loss = 0
                model.encoder.train()
                model.generator.train()
                model.marnn.train()
                model.dis.train()
                torch.cuda.synchronize()
                train_it_start = int(round(time.time()*1000))
                for data in tqdm(self.train_dataloader):
                    
                    input_data = data['x'].to(self.device)
                    previous_y = input_data[:,-1,0].to(device)
                    target = data['y'].to(self.device)
                    
                    model.encoder.zero_grad()
                    model.generator.zero_grad()
                    latent_space = model.encoder(input_data)
                    y_fake = model.generator(latent_space, previous_y)
                    #train discriminator
                    model.dis.zero_grad()
#                     y_fake = model.marnn(input_data,previous_y)
#                     print(y_fake, target)
                    d_out_fake = model.dis(y_fake)
                    d_out_real = model.dis(target)
#                     print("Fake discriminator", d_out_fake)
#                     print("Real discriminator",d_out_real)
                    # train discriminator with fake samples
                    d_loss_fake = criterion(d_out_fake, self.fake_labels)
                    d_loss_fake.backward()
#                     optimizer_dis.step()

                    # train discriminator with true samples
                    d_loss_real = criterion(d_out_real, self.true_labels)
                    d_loss_real.backward()
#                     optimizer_dis.step()

                    d_loss = d_loss_fake + d_loss_real
                    epoch_trainD_loss  += d_loss.item()
                    optimizer_dis.step()
                    
                    # Gradient clipping
                    with torch.no_grad():
                        for param in model.dis.parameters():
                            param.clamp_(-0.01, 0.01)
#                     for name, param in model.named_parameters():
#                         print(name, param)

                    #train generator
                    model.generator.zero_grad()
                    y_fake = model.generator(latent_space,previous_y)
                    dis_out = model.dis(y_fake)
                    g_loss = criterion(dis_out, self.true_labels)
                    g_loss.backward()

                    epoch_trainG_loss += g_loss.item()
                    nn.utils.clip_grad_norm_(model.generator.parameters(), clip)

                    optimizer_generator.step()

                    #train marnn
                    model.marnn.zero_grad()
                    y_pred = model.marnn(input_data,previous_y)
                    marnn_loss = criterion(y_pred,target)
                    marnn_loss.backward()
                    epoch_train_marnn_loss += marnn_loss.item()
#                     print("G", marnn_loss.item())
                    nn.utils.clip_grad_norm_(model.marnn.parameters(), clip)
                    optimizer_marnn_t.step()
                
                trainD_loss = epoch_trainD_loss / len(self.train_dataloader)
                trainG_loss = epoch_trainG_loss / len(self.train_dataloader)
                train_marnn_loss = epoch_train_marnn_loss / len(self.train_dataloader)
                torch.cuda.synchronize()
                time_elapsed = int(round(time.time()*1000)) - train_it_start
                total_train_time += time_elapsed
                
                print('Train epoch {}: Discriminator train loss: {:.6f}\tGenerator train loss: {:.6f}\tTrain marnn Loss: {:.6f}'.format(epoch,trainD_loss,trainG_loss,train_marnn_loss))
            #validation
            model.eval()
            epoch_val_loss = 0
            with torch.no_grad():
                for data in tqdm(self.valid_dataloader):
                    batch_loss =  0
                    input = data['x'].to(self.device)
                    previous_y = input[:,-1,0].to(device)
                    target = data['y'].to(self.device)
#                     print(target.shape)
                    outputs = model(input,previous_y)
                    batch_loss = criterion(outputs,target)
                    epoch_val_loss += batch_loss.item()
                val_loss = epoch_val_loss / len(self.valid_dataloader)
                val_losses.append(val_loss)
                scheduler.step(val_loss)
                print('Valid loss : {}'.format(val_loss))
            
            if self.stop_until_reach_accuracy:
                es(train_r2_loss, model)
            else:
                es(val_loss, model)


#         model_utils.save_checkpoint(model, optimizer_marnn, self._weights_path)
        return val_losses[-1], total_train_time
            
    def test(self):
        self._model.load_state_dict(torch.load(self._weights_path)["model_dict"])
        model = self._model
        model.eval
        groundtruth = []
        predict = [] 
        lst_inference_time = []
        with torch.no_grad():
            i = 0
            for data in tqdm(self.test_dataloader):
                torch.cuda.synchronize()
                inference_time_start = int(round(time.time()*1000))
                if (i%self._output_size) == 0:
                    x = data['x'].to(self.device)
                    y = data['y'].to(self.device)
                    target = data['target'].to(self.device)
                    next = data['next'].to(self.device)
                    for j in range(self._output_size):
                        output = model.marnn(x,target)
                        # import pdb; pdb.set_trace()
                        pred_targets = target[:,1:self.window_size]
                        target[:,0:self.window_size-1] = pred_targets
                        target[:,self.window_size-1] = output[:,0]
                        x_next = x[:,1:self.window_size,:]
                        x[:,0:self.window_size - 1,:] = x_next
                        x[:,self.window_size -1,:] = next[:,j,:]
                        x[:,self.window_size -1,0] = output[:,0]


                        groundtruth_final = y[:,j].view(self._batch_size,1).view(-1, 1).squeeze(-1)
                        output_final = output.view(-1, 1).squeeze(-1)

                        groundtruth += groundtruth_final.tolist()
                        predict += output_final.tolist()
                torch.cuda.synchronize()
                total_inference_time = int(round(time.time()*1000)) - inference_time_start 
                lst_inference_time.append(total_inference_time)
                i+=1
                
        predict_ = np.expand_dims(predict, 1)
        groundtruth_ = np.expand_dims(groundtruth, 1)
        predict_cpy_n_cols =  np.repeat(predict_, self.input_dim, axis=1)
        groundtruth_cpy_n_cols = np.repeat(groundtruth_, self.input_dim, axis=1)
        predicts = self._scaler.inverse_transform(predict_cpy_n_cols)[:, [0]]
        groundtruths = self._scaler.inverse_transform(groundtruth_cpy_n_cols)[:,[0]]

        
        final_predicts = predicts.squeeze(-1)
        final_groundtruths = groundtruths.squeeze(-1)
        num_params = sum(p.numel() for p in model.parameters())
        test_time = np.sum(np.array(lst_inference_time))

        # model_utils.save_results(final_groundtruths, final_predicts, self._base_dir)
        # model_utils.visualize(final_groundtruths, final_predicts, self._base_dir, "results.png")
        # model_utils.save_results(final_groundtruths, final_predicts, self._base_dir, self.target_station, self._output_size)
#         model_utils.save_results(final_groundtruths, final_predicts, self._base_dir, self.target_station, num_params, np.mean(np.array(lst_inference_time)))
#         model_utils.visualize(final_groundtruths, final_predicts, self._base_dir, "result_{}.png".format(self.target_station) )


In [171]:
config_path = '../../config/magan_test.yml'
with open(config_path, 'r', encoding = "utf-8") as f:
    config= yaml.safe_load(f)
config['batch_size']
config['lr']

0.001

In [172]:
config_path = '../../config/magan_test.yml'
with open(config_path, 'r', encoding = "utf-8") as f:
    config= yaml.safe_load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

supervisor = MAGANSupervisor(config, target_station = "房山", device = device)
supervisor.train()

  0%|                                                                                            | 0/9 [00:01<?, ?it/s]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [88]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from util.metric import mae, mse, rmse, mape, nse, mdape
supervisor = MAGANSupervisor(config, target_station = "房山", device = device)
supervisor.test()

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]


KeyError: 'target'

### Train như regression bình thường

In [13]:
class MAGANSupervisor():
    def __init__(self, config, target_station, device):
        self._epochs = config["epochs"]
        self._lr = config["lr"]
        self.window_size = config["window_size"]

        self._optimizer = config['optimizer']
        self.optimizer = config['optimizer']

        self.input_size = config['window_size']
        self.input_dim = config['driving_series']

        self._hidden_size = config['hidden_size']
        self._output_size = config['output_size']

        self._batch_size = config['batch_size']

        # Data
        self.target_station = target_station
        transformed_data, scaler = get_data_array(config, target_station)
        self._scaler = scaler 
        self.true_labels = torch.ones(self._batch_size,1).to(device)
        self.fake_labels = torch.zeros(self._batch_size,1).to(device)
        self.device = device

        train_dataloader, valid_dataloader, test_dataloader = get_dataloader(data=transformed_data, config=config)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.test_dataloader = test_dataloader
        first_it = next(iter(train_dataloader))   
        X, y = first_it['x'], first_it['y']

#         self._base_dir = model_utils.generate_log_dir(args)
        self._base_dir = os.path.join(config['base_dir'])
        self._weights_path = os.path.join(self._base_dir, "best.pth")

        self.config = config

        # Model
        self._model = MARNN(config, device).to(self.device)
        self._model.apply(weight_init)

        
        self._es = EarlyStopping(
            patience=self.config['patience'],
            verbose=True,
            delta=0.0,
            path = self._weights_path

        )

        self._es_until_reach_accuracy = EarlyStoppingReachAccuracy(
            patience=self.config['patience'],
            verbose=True,
            delta=0.0,
            path = self._weights_path 

        )
        self.train_ratio = 0.3
        self.stop_until_reach_accuracy = False
#         if args.experimental_mode == 'stop_until_reach_accuracy':
#             self.stop_until_reach_accuracy = True

    def train(self):
        torch.autograd.set_detect_anomaly(True)
        val_losses = []
        clip = 5
        model = self._model
        criterion = torch.nn.MSELoss()
        es = EarlyStopping(
            patience=100,
            verbose=True,
            delta=0.0,
#             path=self._weights_path    
            path = self._weights_path

        )
        total_train_time = 0.0
        if self.optimizer == 'adam':
            optimizer_marnn = torch.optim.Adam(model.parameters(), lr=self._lr)
#             optimizer_dis = torch.optim.Adam(model.dis.parameters(), lr=self._lr)
#             optimizer_marnn_t = torch.optim.Adam(model.marnn.parameters(), lr=self._lr)
        # Train the model
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_marnn, mode='min',
            factor=0.1, patience=3, threshold=0.0001, threshold_mode='abs')

        for epoch in range(self._epochs):
            train_loss = []

            if (not es.early_stop):
#                 epoch_trainD_loss = 0 
#                 epoch_trainG_loss = 0
                model.train()
#                 model.dis.train()
                torch.cuda.synchronize()
                train_it_start = int(round(time.time()*1000))
                for data in tqdm(self.train_dataloader):
                    input_data = data['x'].to(self.device)
                    previous_y = input_data[:,-1,0].to(device)
                    target = data['y'].to(self.device)

                    model.zero_grad()
                    y_pred = model(input_data,previous_y)
                    marnn_loss = criterion(y_pred,target)
                    train_loss.append(marnn_loss.item())
                    marnn_loss.backward()
#                     print("G", marnn_loss.item())
                    nn.utils.clip_grad_norm_(model.parameters(), clip)
                    optimizer_marnn.step()
#                 train_loss = epoch_trainG_loss / len(self.train_dataloader)
                torch.cuda.synchronize()
                time_elapsed = int(round(time.time()*1000)) - train_it_start
                total_train_time += time_elapsed
                
#                 print('Train epoch {}: Discriminator train loss: {:.6f}\tMARNN train loss: {:.6f}\tTrain marnn Loss: {:.6f}'.format(epoch,epoch_trainD_loss,epoch_trainG_loss,marnn_loss.item()))
                print('Train epoch {}: \tMARNN train loss: {:.6f}'.format(epoch,sum(train_loss)/len(train_loss)))

            #validation
            model.eval()
            epoch_val_loss = 0
            with torch.no_grad():
                for data in tqdm(self.valid_dataloader):
                    batch_loss =  0
                    input = data['x'].to(self.device)
                    previous_y = input[:,-1,0].to(device)
                    target = data['y'].to(self.device)
#                     print(target.shape)
                    outputs = model(input,previous_y)
                    batch_loss = criterion(outputs,target)
                    epoch_val_loss += batch_loss.item()
                val_loss = epoch_val_loss / len(self.valid_dataloader)
                val_losses.append(val_loss)
                scheduler.step(val_loss)
                print('Valid loss : {}'.format(val_loss))
            
            if self.stop_until_reach_accuracy:
                es(train_r2_loss, model)
            else:
                es(val_loss, model)


#         model_utils.save_checkpoint(model, optimizer_marnn, self._weights_path)
        return val_losses[-1], total_train_time
            
    def test(self):
        criterion = nn.MSELoss()
        self._model.load_state_dict(torch.load(self._weights_path)["model_dict"])
        model = self._model
        groundtruth = []
        predict = [] 
        lst_inference_time = []
        model.eval()
        
        with torch.no_grad():
            for data in tqdm(self.test_dataloader):
                x = data['x'].to(self.device)
                previous_y = x[:,-1,0].to(device)

                y = data['y'].to(self.device)

                torch.cuda.synchronize()
                inference_time_start = int(round(time.time()*1000))

                output = model(x,previous_y)

                torch.cuda.synchronize()
                total_inference_time = int(round(time.time()*1000)) - inference_time_start 
                lst_inference_time.append(total_inference_time)

                groundtruth_final = y.view(-1, 1).squeeze(-1)
                output_final = output.view(-1, 1).squeeze(-1)

                groundtruth += groundtruth_final.tolist()
                predict += output_final.tolist()

        predict_ = np.expand_dims(predict, 1)
        groundtruth_ = np.expand_dims(groundtruth, 1)
        predict_cpy_n_cols =  np.repeat(predict_, self.input_dim, axis=1)
        groundtruth_cpy_n_cols = np.repeat(groundtruth_, self.input_dim, axis=1)

        predicts = self._scaler.inverse_transform(predict_cpy_n_cols)[:, [0]]
        groundtruths = self._scaler.inverse_transform(groundtruth_cpy_n_cols)[:,[0]]

        final_predicts = predicts.squeeze(-1)
        final_groundtruths = groundtruths.squeeze(-1)
        m_mae = mae( final_groundtruths , final_predicts)
        m_mape = mape( final_groundtruths , final_predicts)
        m_rmse = rmse( final_groundtruths , final_predicts)
        m_mse = mse( final_groundtruths , final_predicts)
        m_r2score = r2_score( final_groundtruths , final_predicts)
        m_mdape = mdape( final_groundtruths , final_predicts)
        print("MAE: ", m_mae)
        print("MAPE: ", m_mape)
        print("RMSE: ", m_rmse)
        print("MSE: ", m_mse)
        print("R2: ", m_r2score)
        print("MDAE: ", m_mdape)


#         model_utils.save_results(final_groundtruths, final_predicts, self._base_dir)
#         model_utils.visualize(final_groundtruths, final_predicts, self._base_dir, "results.png")

#         with torch.no_grad():
#             i = 0

#             for data in tqdm(self.test_dataloader):
#                 torch.cuda.synchronize()
#                 inference_time_start = int(round(time.time()*1000))
#                 if (i%self._output_size) == 0:
#                     input_data = data['x'].to(self.device)
#                     previous_y = input_data[:,-1,0].to(device)
#                     target = data['y'].to(self.device)
#                     output = modet(input_data, previous_y)
#                     print(output)
#                     next = data['next'].to(self.device)
#                     for j in range(self._output_size):
                        
#                         output = model(x,target)
#                         # import pdb; pdb.set_trace()
#                         pred_targets = target[:,1:self.window_size]
#                         target[:,0:self.window_size-1] = pred_targets
#                         target[:,self.window_size-1] = output[:,0]
#                         x_next = x[:,1:self.window_size,:]
#                         x[:,0:self.window_size - 1,:] = x_next
#                         x[:,self.window_size -1,:] = next[:,j,:]
#                         x[:,self.window_size -1,0] = output[:,0]


#                         groundtruth_final = y[:,j].view(self._batch_size,1).view(-1, 1).squeeze(-1)
#                         output_final = output.view(-1, 1).squeeze(-1)

#                         groundtruth += groundtruth_final.tolist()
#                         predict += output_final.tolist()
#                 torch.cuda.synchronize()
#                 total_inference_time = int(round(time.time()*1000)) - inference_time_start 
#                 lst_inference_time.append(total_inference_time)
#                 i+=1
                
#         predict_ = np.expand_dims(predict, 1)
#         groundtruth_ = np.expand_dims(groundtruth, 1)
#         predict_cpy_n_cols =  np.repeat(predict_, self.input_dim, axis=1)
#         groundtruth_cpy_n_cols = np.repeat(groundtruth_, self.input_dim, axis=1)
#         predicts = self._scaler.inverse_transform(predict_cpy_n_cols)[:, [0]]
#         groundtruths = self._scaler.inverse_transform(groundtruth_cpy_n_cols)[:,[0]]

        
#         final_predicts = predicts.squeeze(-1)
#         final_groundtruths = groundtruths.squeeze(-1)
#         num_params = sum(p.numel() for p in model.parameters())
#         test_time = np.sum(np.array(lst_inference_time))

#         model_utils.save_results(final_groundtruths, final_predicts, self._base_dir)
#         model_utils.visualize(final_groundtruths, final_predicts, self._base_dir, "results.png")
        # model_utils.save_results(final_groundtruths, final_predicts, self._base_dir, self.target_station, self._output_size)
#         model_utils.save_results(final_groundtruths, final_predicts, self._base_dir, self.target_station, num_params, np.mean(np.array(lst_inference_time)))
#         model_utils.visualize(final_groundtruths, final_predicts, self._base_dir, "result_{}.png".format(self.target_station) )


In [14]:
config_path = '../../config/magan_test.yml'
with open(config_path, 'r', encoding = "utf-8") as f:
    config= yaml.safe_load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

supervisor = MAGANSupervisor(config, target_station = "房山", device = device)
supervisor.train()

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:30<00:00,  1.63s/it]


Train epoch 0: 	MARNN train loss: 945.031763


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 41.75it/s]


Valid loss : 0.1947392662140456
Validation loss decreased (inf --> 0.194739).  Saving model ...


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:18<00:00,  1.01it/s]


...

Train epoch 19: 	MARNN train loss: 0.008685


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 36.86it/s]


Valid loss : 0.003830655160444704
EarlyStopping counter: 2 out of 100


  5%|████▎                                                                              | 1/19 [00:01<00:33,  1.88s/it]


KeyboardInterrupt: 

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from util.metric import mae, mse, rmse, mape, nse, mdape
supervisor = MAGANSupervisor(config, target_station = "房山", device = device)
supervisor.test()

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 36.57it/s]


MAE:  6.821328729159084
MAPE:  36.27126804665093
RMSE:  10.264101944082617
MSE:  105.35178871852055
R2:  0.9590084283700707
MDAE:  11.598088172249845


In [74]:
config['output_size']

1

In [ ]:
# sửa conv